# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,grytviken,-54.2811,-36.5092,276.62,95,64,4.66,GS,1705001557
1,1,enewetak,11.3474,162.3373,300.14,76,69,10.09,MH,1705001607
2,2,tual,-5.6667,132.7500,302.28,77,93,6.55,ID,1705001621
3,3,stanley,54.8680,-1.6985,277.75,94,75,3.60,GB,1705001523
4,4,talnakh,69.4865,88.3972,257.71,76,100,3.38,RU,1705001587


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
# Configure the map plot
%%capture --no-display
city_map = city_data_df.hvplot.points(
    x='Lng',
    y='Lat',
    c='Humidity',
    hover_cols=['City'],
    size='Humidity',
    title='City Map: Humidity',
    xlabel='Longitude',
    ylabel='Latitude',
    colorbar=True,
    clim=(0, city_data_df['Humidity'].max()),
    cmap='viridis',
    geo=True,
    tiles='CartoLight'
)

# Display the map
city_map


UsageError: Line magic function `%%capture` not found.


### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:

lowest_values = city_data_df.nsmallest(5, 'Max Temp')
lowest_values

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
282,282,susuman,62.7833,148.1667,229.81,100,99,0.45,RU,1705002185
451,451,khandyga,62.6667,135.6000,230.66,100,100,0.47,RU,1705002231
44,44,yellowknife,62.4560,-114.3525,231.44,60,40,2.06,CA,1705001566
84,84,udachny,66.4167,112.4000,233.78,100,73,1.92,RU,1705001574
542,542,tommot,58.9564,126.2925,234.45,99,82,0.41,RU,1705002253


In [8]:
# Ideal weather conditions
ideal_conditions = (city_data_df['Max Temp'] < 500.00) & (city_data_df['Max Temp'] > 250.00) & \
                   (city_data_df['Wind Speed'] < 10.00) & (city_data_df['Cloudiness'] < 40)

# Apply conditions to filter the DataFrame
ideal_cities_df = city_data_df[ideal_conditions]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data of the filtered DataFrame
ideal_cities_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
7,7,ancud,-41.8697,-73.8203,289.71,67,5,3.29,CL,1705001675
11,11,saipan,15.1355,145.7010,298.54,94,0,2.57,MP,1705001608
14,14,colonia,50.9333,6.9500,272.19,77,0,2.06,DE,1705001387
25,25,college,64.8569,-147.8028,252.14,68,20,2.06,US,1705001554
27,27,punta cana,18.5818,-68.4043,300.82,68,6,2.97,DO,1705001905


### Step 3: Create a new DataFrame called `hotel_df`.

In [9]:
# Use the Pandas copy function to create DataFrame called hotel_df
hotel_df = ideal_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame
hotel_df = hotel_df.assign(Hotel_Name="")

# Display sample data
hotel_df.head()


,City,Country,Lat,Lng,Humidity,Hotel_Name
7,ancud,CL,-41.8697,-73.8203,67,
11,saipan,MP,15.1355,145.7010,94,
14,colonia,DE,50.9333,6.9500,77,
25,college,US,64.8569,-147.8028,68,
27,punta cana,DO,18.5818,-68.4043,68,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [12]:
geoapify_key

'89e17f88'

In [11]:
# Set parameters to search for a hotel
radius = 10000  
params = {
    'limit': 20,  
    'categories': 'accommodation.hotel', 
    'apiKey': geoapify_key  
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    lat = row['Lat']
    lon = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params['filter'] = f'circle:{lon},{lat},{radius}'
    params['bias'] = f'proximity:{lon},{lat}'  
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    result = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = result["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()


Starting hotel search
ancud - nearest hotel: No hotel found
saipan - nearest hotel: No hotel found
colonia - nearest hotel: No hotel found
college - nearest hotel: No hotel found
punta cana - nearest hotel: No hotel found
badger - nearest hotel: No hotel found
sarupathar - nearest hotel: No hotel found
blackmans bay - nearest hotel: No hotel found
gumla - nearest hotel: No hotel found
edinburgh of the seven seas - nearest hotel: No hotel found
kapa'a - nearest hotel: No hotel found
tolanaro - nearest hotel: No hotel found
greenville - nearest hotel: No hotel found
boende - nearest hotel: No hotel found
chonchi - nearest hotel: No hotel found
mananjary - nearest hotel: No hotel found
sao vicente - nearest hotel: No hotel found
pacific grove - nearest hotel: No hotel found
trevelin - nearest hotel: No hotel found
longyearbyen - nearest hotel: No hotel found
vila velha - nearest hotel: No hotel found
bangassou - nearest hotel: No hotel found
newman - nearest hotel: No hotel found
qamdo - 

KeyboardInterrupt: 

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display



In [ ]:
# Configure the map plot with additional hover information
city_map_with_hotels = hotel_df.hvplot.points(
    x='Longitude',
    y='Latitude',
    c='Humidity',
    hover_cols=['City', 'Country', 'Hotel Name'],  # Add 'Country' and 'Hotel Name' to the hover information
    size='Humidity',
    title='City Map with Humidity and Hotels',
    xlabel='Longitude',
    ylabel='Latitude',
    colorbar=True,
    clim=(0, hotel_df['Humidity'].max()),
    cmap='viridis',
    geo=True,
    tiles='Stamen Terrain'
)

# Display the map
city_map_with_hotels
